### 2016 - 2020 Presidential Election Comparison

In [1]:
# Imports
import os
import pandas as pd
from zipfile import ZipFile

In [2]:
def print_df_overview(df, title):        
    print(f"{title} DF Head:")
    print(df.head())
    print("----------------------------------")
    print(f"{title} DF dtypes:")
    print(df.dtypes)
    
    for col in df.columns:
        print("----------------------------------")
        print(f"{title} DF column value counts:")

        ABRIDGED_ROWS = 16
        counts = df[col].value_counts()
        if counts.size < 2*ABRIDGED_ROWS:
            # Print the counts all together
            print(col, "counts:")    
            print(counts)
        else:
            # Print top and bottom counts
            print(col, "top counts:")
            print(counts[0:ABRIDGED_ROWS])
            print(col, "bottom counts:")
            print(counts[-ABRIDGED_ROWS:])

In [3]:
# The data files are too big to naively manage in git, so they are stored as zip files.
def unzip(zip, where):
    with ZipFile(zip, 'r') as zObject:   
        print(f"Unzipping {zip} to {where}")
        zObject.extractall(path=where) 
        
path_2016 = os.path.join('.', 'resources', '2016')
path_2016_csv = os.path.join('.', 'resources', '2016', '2016-precinct-president.csv')
path_2016_zip = os.path.join('.', 'resources', '2016', '2016-precinct-president.csv.zip')
path_2020 = os.path.join('.', 'resources', '2020')
path_2020_csv = os.path.join('.', 'resources', '2020', 'PRESIDENT_precinct_general.csv')
path_2020_zip = os.path.join('.', 'resources', '2020', 'PRESIDENT_precinct_general.csv.zip')

unzip(path_2016_zip, path_2016)
unzip(path_2020_zip, path_2020)

Unzipping .\resources\2016\2016-precinct-president.csv.zip to .\resources\2016
Unzipping .\resources\2020\PRESIDENT_precinct_general.csv.zip to .\resources\2020


In [16]:
# Read raw data -- takes several seconds

# Encoding wasn't documented but ISO-8859-1 seems to work fine. 
# Some data types specified here to suppress warnings -- they're actually provided for 2020
raw_2016_df = pd.read_csv(path_2016_csv, encoding="ISO-8859-1", 
                          dtype={"precinct": str, "district": str, "party": str, "candidate_fec": str, "candidate_fec_name": str} )

# These come straight from the data documentation:
official_2020_dtypes = {'precinct':str,'office':str, 'party_detailed':str, 
		'party_simplified':str,'mode':str,'votes':int, 'county_name':str,
		'county_fips':str, 'jurisdiction_name':str,'jurisdiction_fips':str,
		'candidate':str, 'district':str, 'dataverse':str,'year':int,
		'stage':str, 'state':str, 'special':str, 'writein':str, 'state_po':str,
		'state_fips':str, 'state_cen':str, 'state_ic':str, 'date':str, 
		'readme_check':str,'magnitude':int}
raw_2020_df = pd.read_csv(path_2020_csv, encoding="ISO-8859-1", dtype=official_2020_dtypes )

raw_2016_df.shape, raw_2020_df.shape

((1989234, 37), (1982581, 25))

In [22]:
# For easier work during development, limit data to the smallest state
#raw_2016_df = raw_2016_df.loc[raw_2016_df["state"] == "RHODE ISLAND"]
#raw_2020_df = raw_2020_df.loc[raw_2020_df["state"] == "Rhode Island"]
raw_2016_df = raw_2016_df.loc[raw_2016_df["state"] == "Rhode Island"]
raw_2020_df = raw_2020_df.loc[raw_2020_df["state"] == "RHODE ISLAND"]

# These don't impact flow or anything like that, just print some info about the DFs to inform cleaning
#print_df_overview(raw_2016_df, "2016 Raw")
#print_df_overview(raw_2020_df, "2020 Raw")

2016 Raw DF Head:
         year stage  special         state state_postal  state_fips  \
1689923  2016   gen    False  Rhode Island           RI          44   
1689924  2016   gen    False  Rhode Island           RI          44   
1689925  2016   gen    False  Rhode Island           RI          44   
1689926  2016   gen    False  Rhode Island           RI          44   
1689927  2016   gen    False  Rhode Island           RI          44   

         state_icpsr county_name  county_fips  county_ansi  ...  \
1689923            5         NaN          NaN          NaN  ...   
1689924            5         NaN          NaN          NaN  ...   
1689925            5         NaN          NaN          NaN  ...   
1689926            5         NaN          NaN          NaN  ...   
1689927            5         NaN          NaN          NaN  ...   

         candidate_middle  candidate_full candidate_suffix candidate_nickname  \
1689923               NaN             NaN              NaN             

In [23]:
# Get ready to clean up
c_2016_df = raw_2016_df.copy()
c_2020_df = raw_2020_df.copy()
print("Initial:                 ", c_2016_df.shape, c_2020_df.shape)

# Results include "statistial adjustments" which I'm ignoring for the moment
c_2016_df = c_2016_df.drop(c_2016_df[c_2016_df.precinct == "Statistical Adjustments"].index)
c_2020_df = c_2020_df.drop(c_2020_df[c_2020_df.jurisdiction_name == "{STATISTICAL ADJUSTMENTS}"].index)
print("Drop adjustments:        ", c_2016_df.shape, c_2020_df.shape)

# Also remove any negative votes -- I don't know what they mean
c_2016_df = c_2016_df.drop(c_2016_df[c_2016_df.votes <0].index)
c_2020_df = c_2020_df.drop(c_2020_df[c_2020_df.votes <0].index)
print("Drop negative votes:     ", c_2016_df.shape, c_2020_df.shape)

# Narrow down to the interesting columns.

#c_2016_df = c_2016_df.loc[:, ["state", "county_name", "jurisdiction", "candidate", "votes"] ]
c_2016_df = c_2016_df.loc[:, ["state", "county_name", "candidate", "votes"] ]
#c_2020_df = c_2020_df.loc[:, ["state", "county_name", "jurisdiction_name", "candidate", "votes"] ]
c_2020_df = c_2020_df.loc[:, ["state", "county_name", "candidate", "votes"] ]
print("Drop extraneous columns: ", c_2016_df.shape, c_2020_df.shape)

# Change a couple column names for consistency and conciseness
c_2016_df.rename(columns={"county_name": "county"}, inplace=True)
c_2020_df.rename(columns={"county_name": "county"}, inplace=True)
#c_2020_df.rename(columns={"jurisdiction_name": "jurisdiction"}, inplace=True)

# 2016 uses normal capitalization while 2020 uses all caps
# So, California vs CALIFORNIA, and District of Columbia vs DISTRICT OF COLUMBIA
# To reconcile them, force everything to lowercase.
c_2016_df["state"] = c_2016_df["state"].str.lower()
c_2016_df["county"] = c_2016_df["county"].str.lower()
#c_2016_df["jurisdiction"] = c_2016_df["jurisdiction"].str.lower()
c_2016_df["candidate"] = c_2016_df["candidate"].str.lower()
c_2020_df["state"] = c_2020_df["state"].str.lower()
c_2020_df["county"] = c_2020_df["county"].str.lower()
#c_2020_df["jurisdiction"] = c_2020_df["jurisdiction"].str.lower()
c_2020_df["candidate"] = c_2020_df["candidate"].str.lower()

# County names actually say "county" in the 2016 data, so remove that
c_2016_df["county"] = c_2016_df["county"].str.replace(" county", "")

print("2016:")
raw_rows = raw_2016_df.shape[0]
raw_votes = raw_2016_df['votes'].sum()
clean_rows = c_2016_df.shape[0]
clean_votes = c_2016_df['votes'].sum()
print(f"Original: {raw_rows:7} rows, {raw_votes:10} votes")
print(f"Cleaned:  {clean_rows:7} rows, {clean_votes:10} votes -- {clean_rows / raw_rows:.3%} and {clean_votes / raw_votes:.3%}")

print("2020:")
raw_rows = raw_2020_df.shape[0]
raw_votes = raw_2020_df['votes'].sum()
clean_rows = c_2020_df.shape[0]
clean_votes = c_2020_df['votes'].sum()
print(f"Original: {raw_rows:7} rows, {raw_votes:10} votes")
print(f"Cleaned:  {clean_rows:7} rows, {clean_votes:10} votes -- {clean_rows / raw_rows:.3%} and {clean_votes / raw_votes:.3%}")

Initial:                  (6396, 37) (10521, 25)
Drop adjustments:         (6396, 37) (10521, 25)
Drop negative votes:      (6396, 37) (10521, 25)
Drop extraneous columns:  (6396, 4) (10521, 4)
2016:
Original:    6396 rows,     464144 votes
Cleaned:     6396 rows,     464144 votes -- 100.000% and 100.000%
2020:
Original:   10521 rows,     517757 votes
Cleaned:    10521 rows,     517757 votes -- 100.000% and 100.000%


In [24]:
#print_df_overview(c_2016_df, "2016 Cleaned")
#print_df_overview(c_2020_df, "2020 Cleaned")
c_2016_df, c_2020_df

(                state      county           candidate  votes
 1689923  rhode island         NaN                 NaN      0
 1689924  rhode island         NaN                 NaN      1
 1689925  rhode island         NaN     hillary clinton      0
 1689926  rhode island         NaN     hillary clinton    376
 1689927  rhode island         NaN  rocky de la fuente      0
 ...               ...         ...                 ...    ...
 1696314  rhode island  washington          jill stein      0
 1696315  rhode island  washington          jill stein      0
 1696316  rhode island  washington          jill stein      0
 1696317  rhode island  washington        donald trump     68
 1696318  rhode island  washington        donald trump      3
 
 [6396 rows x 4 columns],
                 state      county                    candidate  votes
 1011433  rhode island     bristol                      writein      3
 1011434  rhode island     bristol                      writein      0
 1011435  rhode

In [26]:
# Combine the dataframes, with each year getting its own column for votes
# This isn't quite working. I'm pretty sure I need a join, not a merge, but I'm getting sleepy...
combo_df = pd.merge(c_2016_df, c_2020_df, on=["state", "county"])
print_df_overview(combo_df, "Combo DF")

Combo DF DF Head:
          state county candidate_x  votes_x     candidate_y  votes_y
0  rhode island    NaN         NaN        0         writein        0
1  rhode island    NaN         NaN        0         writein        0
2  rhode island    NaN         NaN        0         writein        3
3  rhode island    NaN         NaN        0  joseph r biden        0
4  rhode island    NaN         NaN        0  joseph r biden        0
----------------------------------
Combo DF DF dtypes:
state          object
county         object
candidate_x    object
votes_x         int64
candidate_y    object
votes_y         int32
dtype: object
----------------------------------
Combo DF DF column value counts:
state counts:
state
rhode island    25972947
Name: count, dtype: int64
----------------------------------
Combo DF DF column value counts:
county counts:
county
providence    22561812
kent           1683360
washington     1066338
newport         517440
bristol         143451
Name: count, dtype: int